In [13]:
import tensorflow as tf
from tensorflow import keras
import time, os

In [14]:
start = time.time() # 시작 시간 저장

In [15]:
def get_dataset():
    batch_size = 32
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a `tf.data.Dataset`.
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )
#end of def:


In [16]:
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model
#end of def:

In [17]:
# Create a MirroredStrategy.
print(tf.config.list_physical_devices("CPU"))
print(tf.config.list_physical_devices("GPU"))
gpus = tf.config.experimental.list_logical_devices("GPU")

if len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('\n\nRunning on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('\n\nRunning on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print("'\n\nRunning on CPU '")
#end of if
 
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[]
'

Running on CPU '
Number of devices: 1
Epoch 1/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.2263 - sparse_categorical_accuracy: 0.9327 - val_loss: 0.1342 - val_sparse_categorical_accuracy: 0.9596
Epoch 2/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.0940 - sparse_categorical_accuracy: 0.9711 - val_loss: 0.0979 - val_sparse_categorical_accuracy: 0.9700
Epoch 3/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.0593 - sparse_categorical_accuracy: 0.9816 - val_loss: 0.0917 - val_sparse_categorical_accuracy: 0.9729
Epoch 4/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.0412 - sparse_categorical_accuracy: 0.9868 - val_loss: 0.0916 - val_sparse_categorical_accuracy: 0.9749
Epoch 5/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.0346 - sparse_categorical_accuracy: 0.9883 - val_loss: 0.0919 - val_sparse_categori

[0.09778446704149246, 0.9800000190734863]

In [18]:
print("실행시간 :", round(time.time() - start, 2),"(초)")  # 현재시각 - 시작시간 = 실행 시간

실행시간 : 63.14 (초)


CPU:

실행시간 : 63.14 (초)

GPU:

실행시간 : 40.48 (초)